# Advanced Querying Mongo

Importing libraries and setting up connection

In [ ]:
from pymongo import MongoClient
import re
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

In [ ]:
list(db.list_collections())

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [ ]:
query = {"name":"Babelgum"}
list(db["companies"].find(query, projection={"name"}))


### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [ ]:
query2 = {"number_of_employees":{"$gt":5000}}
list(db["companies"].find(query2, projection={"name", "number_of_employees"}).limit(20).sort("number_of_employees", -1))

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [ ]:
query3 = {"founded_year":{"$gte":2000, "$lte":2000}}
list(db["companies"].find(query3, projection={"name", "founded_year"}))

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [ ]:
query4 = {"$and":[{"ipo.valuation_amount":{"$gt":100000000}}, {"founded_year":{"$lt":2010}}]}
list(db["companies"].find(query4, projection={"name", "ipo"}))

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [ ]:
query5 = {"$and":[{"number_of_eployees":{"$lt":1000}}, {"founded_year":{"$lt":2005}}]}
list(db["companies"].find(query4, projection={"name"}).limit(10).sort("number_of_employees", -1))

### 6. All the companies that don't include the `partners` field.

In [ ]:
#Todos los que no tengan el campo

query6 = {"$and":[{"partners":{"$exists": False}}]}
list(db["companies"].find(query6, projection={"name"}))

In [ ]:
#Todos los que tengan el campo pero esté vacío

query6 = {"$and":[{"partners":{"$exists": True, "$not": {"$size": 0}}}]}
list(db["companies"].find(query6, projection={"name"}))

### 7. All the companies that have a null type of value on the `category_code` field.

In [ ]:
query7 = {"category_code":None}
list(db["companies"].find(query7, projection={"name"}))

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [ ]:
query8 = {"number_of_employees":{"$gte":100, "$lt":1000}}
list(db["companies"].find(query8, projection={"name", "number_of_employees"}))

### 9. Order all the companies by their IPO price in a descending order.

In [ ]:
query9 = {"ipo.valuation_amount": {'$exists': True}}
list(db['companies'].find(query9).sort([("ipo.valuation_amount",-1)]))

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [ ]:
query10 = {"number_of_employees":{"$exists":True}}
list(db["companies"].find(query10, projection={"name"}).sort([("number_of_employees", 1)]).limit(10))

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [ ]:
query11 = {"founded_month": {'$gte':7 ,'$lte':12}}
list(db['companies'].find(query11).sort([("number_of_employees",1)]).limit(1000))

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [ ]:
query12 = {"founded_year": {'$lt':2000}, "acquisition.price_amount":{"$gt":1000000}}
list(db['companies'].find(query12))

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [ ]:

query13 = {"acquisition.acquired_year": {'$gt':2010}}
list(db['companies'].find(query13, {"item": 1, "name": 1, "acquisition":1,"_id": 0}).sort([("acquisition.price_amount",1)]))

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [ ]:
query14 = {"founded_year": { '$exists': True, '$not': {'$size': 0} }}
list(db['companies'].find(query14, {"item": 1, "name": 1, "acquisition":1,"_id": 0}).sort([("founded_year",1)]).limit(100))

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [ ]:
query15 = {"founded_day": {'$lte': 7}}
solution15 = db['companies'].find(query15).sort([("acquisition.price_amount",1)]).limit(10)

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [ ]:
query16 = {'category_code': {'$eq': 'web'}, 'number_of_employees':{'$gt': 4000}}
list(db['companies'].find(query16).sort([("number_of_employees",1)]))

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [ ]:
query17 = {"$and":[{"acquisition.price_amount":{"$gt":10000000}}, {"acquisition.price_currency_code":{"$eq":"EUR"}}]}
list(db["companies"].find(query17))

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [ ]:
query18 = {"acquisition.acquired_month": {'$lte': 3}}
list(db['companies'].find(query18, {"item": 1, "name": 1, "acquisition":1,"_id": 0}).limit(10))

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code